# Serverless Task Migration
This app is designed to help identify tasks running in your snowflake that are underutilizing a warehouse so that they can be changed to a serverless task. This is designed to migrate these tasks to improve the proficiency of these services as well as save on the cost of execution, as an under-utilized warehouse often charges for downtime when spun up for short tasks

This Notedbook will drop a streamlit file at SIT_SOLUTIONS.STM.CODE_STAGE and install the app SIT_SOLUTIONS.STM.STM_STREAMLIT.

In [ ]:
# Import python packages
import streamlit as st
import pandas as pd
import pickle
import codecs
import os
import base64

from snowflake.snowpark.context import get_active_session
session = get_active_session()
session.sql(f"""ALTER SESSION SET QUERY_TAG = '{{"origin":"sf_sit","name":"sit_serveless_task_migration","version":{{"major":1, "minor":0}} }}'""").collect()

current_database = session.sql("SELECT CURRENT_DATABASE()").collect()[0][0]
current_schema = session.sql("SELECT CURRENT_schema()").collect()[0][0]
try:
    # session.sql("SELECT * from INSTALL_TEST.TEST.TEST_OPERATIONS").collect()
    st.session_state.opps_test_info = session.sql(f"SELECT * from {current_database}.{current_schema}.TEST_OPERATIONS").to_pandas()
    run_mode = "test"
except:
    st.success("Running Customer Install. Please follow the steps!")
    run_mode = "customer"

Please choose the location you'd like to install to

In [ ]:
if run_mode == 'test':
    opps_info = st.session_state.opps_test_info
    install_db = opps_info[opps_info["KEY"] == "INSTALL_DB"]["VALUE"].values[0]
    admin_schema = opps_info[opps_info["KEY"] == "ADMIN_SCHEMA"]["VALUE"].values[0]
    results_schema = opps_info[opps_info["KEY"] == "RESULT_SCHEMA"]["VALUE"].values[0]
    ops_schema = opps_info[opps_info["KEY"] == "OPS_SCHEMA"]["VALUE"].values[0]
    install_stage = opps_info[opps_info["KEY"] == "INSTALL_STAGE"]["VALUE"].values[0]
    ops_wh = opps_info[opps_info["KEY"] == "OPS_WH"]["VALUE"].values[0]
elif run_mode =="customer":
    install_db = st.text_input("Database")
    admin_schema = st.text_input("Admin Schema")
    results_schema = st.text_input("Results Schema")
    ops_schema = st.text_input("Operations Schema")
    install_stage = st.text_input("Install Stage (for hosting code files)")
    ops_wh = st.text_input("User Warehouse")

In [ ]:
CREATE OR REPLACE PROCEDURE {{install_db}}.{{admin_schema}}.PUT_TO_STAGE(STAGE VARCHAR, FILENAME VARCHAR, CONTENT VARCHAR)
RETURNS STRING
LANGUAGE PYTHON
RUNTIME_VERSION=3.8
PACKAGES=('snowflake-snowpark-python')
HANDLER='put_to_stage'
COMMENT='{"origin": "sf_sit","name": "sit_serveless_task_migration","version": "{major: 1, minor: 0}"}'
AS $$
import io
import os

def put_to_stage(session, stage, filename, content):
    local_path = '/tmp'
    local_file = os.path.join(local_path, filename)
    f = open(local_file, "w", encoding='utf-8')
    f.write(content)
    f.close()
    session.file.put(local_file, '@'+stage, auto_compress=False, overwrite=True)
    return "saved file "+filename+" in stage "+stage
$$;


In [ ]:
"""gASVAR8AAAAAAAB9lIwFZmlsZXOUfZQojAguZ2l0a2VlcJRDAJSMDWluc3RhbGwuaXB5bmKUQs0e
AAB7CiAiY2VsbHMiOiBbCiAgewogICAiY2VsbF90eXBlIjogIm1hcmtkb3duIiwKICAgImlkIjog
IjAxNTE1ZjBjLTM5NzEtNDRiNS04YTg2LTllNWM2MjAyZWJiYSIsCiAgICJtZXRhZGF0YSI6IHsK
ICAgICJjb2xsYXBzZWQiOiBmYWxzZSwKICAgICJuYW1lIjogImNlbGwxMyIKICAgfSwKICAgInNv
dXJjZSI6IFsKICAgICIjIFNlcnZlcmxlc3MgVGFzayBNaWdyYXRpb25cbiIsCiAgICAiVGhpcyBh
cHAgaXMgZGVzaWduZWQgdG8gaGVscCBpZGVudGlmeSB0YXNrcyBydW5uaW5nIGluIHlvdXIgc25v
d2ZsYWtlIHRoYXQgYXJlIHVuZGVydXRpbGl6aW5nIGEgd2FyZWhvdXNlIHNvIHRoYXQgdGhleSBj
YW4gYmUgY2hhbmdlZCB0byBhIHNlcnZlcmxlc3MgdGFzay4gVGhpcyBpcyBkZXNpZ25lZCB0byBt
aWdyYXRlIHRoZXNlIHRhc2tzIHRvIGltcHJvdmUgdGhlIHByb2ZpY2llbmN5IG9mIHRoZXNlIHNl
cnZpY2VzIGFzIHdlbGwgYXMgc2F2ZSBvbiB0aGUgY29zdCBvZiBleGVjdXRpb24sIGFzIGFuIHVu
ZGVyLXV0aWxpemVkIHdhcmVob3VzZSBvZnRlbiBjaGFyZ2VzIGZvciBkb3dudGltZSB3aGVuIHNw
dW4gdXAgZm9yIHNob3J0IHRhc2tzXG4iLAogICAgIlxuIiwKICAgICJUaGlzIE5vdGVkYm9vayB3
aWxsIGRyb3AgYSBzdHJlYW1saXQgZmlsZSBhdCBTSVRfU09MVVRJT05TLlNUTS5DT0RFX1NUQUdF
IGFuZCBpbnN0YWxsIHRoZSBhcHAgU0lUX1NPTFVUSU9OUy5TVE0uU1RNX1NUUkVBTUxJVC4iCiAg
IF0KICB9LAogIHsKICAgImNlbGxfdHlwZSI6ICJjb2RlIiwKICAgImV4ZWN1dGlvbl9jb3VudCI6
IG51bGwsCiAgICJpZCI6ICIzNzc1OTA4Zi1jYTM2LTQ4NDYtOGYzOC01YWRjYTM5MjE3ZjIiLAog
ICAibWV0YWRhdGEiOiB7CiAgICAiY29kZUNvbGxhcHNlZCI6IGZhbHNlLAogICAgImNvbGxhcHNl
ZCI6IGZhbHNlLAogICAgImxhbmd1YWdlIjogInB5dGhvbiIsCiAgICAibmFtZSI6ICJzZXR1cCIs
CiAgICAicmVzdWx0SGVpZ2h0IjogMAogICB9LAogICAib3V0cHV0cyI6IFtdLAogICAic291cmNl
IjogWwogICAgIiMgSW1wb3J0IHB5dGhvbiBwYWNrYWdlc1xuIiwKICAgICJpbXBvcnQgc3RyZWFt
bGl0IGFzIHN0XG4iLAogICAgImltcG9ydCBwYW5kYXMgYXMgcGRcbiIsCiAgICAiaW1wb3J0IHBp
Y2tsZVxuIiwKICAgICJpbXBvcnQgY29kZWNzXG4iLAogICAgImltcG9ydCBvc1xuIiwKICAgICJp
bXBvcnQgYmFzZTY0XG4iLAogICAgIlxuIiwKICAgICJmcm9tIHNub3dmbGFrZS5zbm93cGFyay5j
b250ZXh0IGltcG9ydCBnZXRfYWN0aXZlX3Nlc3Npb25cbiIsCiAgICAic2Vzc2lvbiA9IGdldF9h
Y3RpdmVfc2Vzc2lvbigpXG4iLAogICAgInNlc3Npb24uc3FsKGZcIlwiXCJBTFRFUiBTRVNTSU9O
IFNFVCBRVUVSWV9UQUcgPSAne3tcIm9yaWdpblwiOlwic2Zfc2l0XCIsXCJuYW1lXCI6XCJzaXRf
c2VydmVsZXNzX3Rhc2tfbWlncmF0aW9uXCIsXCJ2ZXJzaW9uXCI6e3tcIm1ham9yXCI6MSwgXCJt
aW5vclwiOjB9fSB9fSdcIlwiXCIpLmNvbGxlY3QoKVxuIiwKICAgICJcbiIsCiAgICAiY3VycmVu
dF9kYXRhYmFzZSA9IHNlc3Npb24uc3FsKFwiU0VMRUNUIENVUlJFTlRfREFUQUJBU0UoKVwiKS5j
b2xsZWN0KClbMF1bMF1cbiIsCiAgICAiY3VycmVudF9zY2hlbWEgPSBzZXNzaW9uLnNxbChcIlNF
TEVDVCBDVVJSRU5UX3NjaGVtYSgpXCIpLmNvbGxlY3QoKVswXVswXVxuIiwKICAgICJ0cnk6XG4i
LAogICAgIiAgICAjIHNlc3Npb24uc3FsKFwiU0VMRUNUICogZnJvbSBJTlNUQUxMX1RFU1QuVEVT
VC5URVNUX09QRVJBVElPTlNcIikuY29sbGVjdCgpXG4iLAogICAgIiAgICBzdC5zZXNzaW9uX3N0
YXRlLm9wcHNfdGVzdF9pbmZvID0gc2Vzc2lvbi5zcWwoZlwiU0VMRUNUICogZnJvbSB7Y3VycmVu
dF9kYXRhYmFzZX0ue2N1cnJlbnRfc2NoZW1hfS5URVNUX09QRVJBVElPTlNcIikudG9fcGFuZGFz
KClcbiIsCiAgICAiICAgIHJ1bl9tb2RlID0gXCJ0ZXN0XCJcbiIsCiAgICAiZXhjZXB0OlxuIiwK
ICAgICIgICAgc3Quc3VjY2VzcyhcIlJ1bm5pbmcgQ3VzdG9tZXIgSW5zdGFsbC4gUGxlYXNlIGZv
bGxvdyB0aGUgc3RlcHMhXCIpXG4iLAogICAgIiAgICBydW5fbW9kZSA9IFwiY3VzdG9tZXJcIiIK
ICAgXQogIH0sCiAgewogICAiY2VsbF90eXBlIjogIm1hcmtkb3duIiwKICAgImlkIjogIjBhMmFi
OTg1LWFlZTEtNGU5NS04NDFiLTNjOTg0MGU0NmE1MyIsCiAgICJtZXRhZGF0YSI6IHsKICAgICJj
b2xsYXBzZWQiOiBmYWxzZSwKICAgICJuYW1lIjogInNldHVwX3Byb21wdCIsCiAgICAicmVzdWx0
SGVpZ2h0IjogNDEKICAgfSwKICAgInNvdXJjZSI6IFsKICAgICJQbGVhc2UgY2hvb3NlIHRoZSBs
b2NhdGlvbiB5b3UnZCBsaWtlIHRvIGluc3RhbGwgdG8iCiAgIF0KICB9LAogIHsKICAgImNlbGxf
dHlwZSI6ICJjb2RlIiwKICAgImV4ZWN1dGlvbl9jb3VudCI6IG51bGwsCiAgICJpZCI6ICI1MTMw
NGE3Yi1lMTE4LTRlYmUtOGRhYy00NzU4MWYwNWVmZTUiLAogICAibWV0YWRhdGEiOiB7CiAgICAi
Y29kZUNvbGxhcHNlZCI6IGZhbHNlLAogICAgImNvbGxhcHNlZCI6IGZhbHNlLAogICAgImxhbmd1
YWdlIjogInB5dGhvbiIsCiAgICAibmFtZSI6ICJtb2RlX2dhdGUiLAogICAgInJlc3VsdEhlaWdo
dCI6IDExMgogICB9LAogICAib3V0cHV0cyI6IFtdLAogICAic291cmNlIjogWwogICAgImlmIHJ1
bl9tb2RlID09ICd0ZXN0JzpcbiIsCiAgICAiICAgIG9wcHNfaW5mbyA9IHN0LnNlc3Npb25fc3Rh
dGUub3Bwc190ZXN0X2luZm9cbiIsCiAgICAiICAgIGluc3RhbGxfZGIgPSBvcHBzX2luZm9bb3Bw
c19pbmZvW1wiS0VZXCJdID09IFwiSU5TVEFMTF9EQlwiXVtcIlZBTFVFXCJdLnZhbHVlc1swXVxu
IiwKICAgICIgICAgYWRtaW5fc2NoZW1hID0gb3Bwc19pbmZvW29wcHNfaW5mb1tcIktFWVwiXSA9
PSBcIkFETUlOX1NDSEVNQVwiXVtcIlZBTFVFXCJdLnZhbHVlc1swXVxuIiwKICAgICIgICAgcmVz
dWx0c19zY2hlbWEgPSBvcHBzX2luZm9bb3Bwc19pbmZvW1wiS0VZXCJdID09IFwiUkVTVUxUX1ND
SEVNQVwiXVtcIlZBTFVFXCJdLnZhbHVlc1swXVxuIiwKICAgICIgICAgb3BzX3NjaGVtYSA9IG9w
cHNfaW5mb1tvcHBzX2luZm9bXCJLRVlcIl0gPT0gXCJPUFNfU0NIRU1BXCJdW1wiVkFMVUVcIl0u
dmFsdWVzWzBdXG4iLAogICAgIiAgICBpbnN0YWxsX3N0YWdlID0gb3Bwc19pbmZvW29wcHNfaW5m
b1tcIktFWVwiXSA9PSBcIklOU1RBTExfU1RBR0VcIl1bXCJWQUxVRVwiXS52YWx1ZXNbMF1cbiIs
CiAgICAiICAgIG9wc193aCA9IG9wcHNfaW5mb1tvcHBzX2luZm9bXCJLRVlcIl0gPT0gXCJPUFNf
V0hcIl1bXCJWQUxVRVwiXS52YWx1ZXNbMF1cbiIsCiAgICAiZWxpZiBydW5fbW9kZSA9PVwiY3Vz
dG9tZXJcIjpcbiIsCiAgICAiICAgIGluc3RhbGxfZGIgPSBzdC50ZXh0X2lucHV0KFwiRGF0YWJh
c2VcIilcbiIsCiAgICAiICAgIGFkbWluX3NjaGVtYSA9IHN0LnRleHRfaW5wdXQoXCJBZG1pbiBT
Y2hlbWFcIilcbiIsCiAgICAiICAgIHJlc3VsdHNfc2NoZW1hID0gc3QudGV4dF9pbnB1dChcIlJl
c3VsdHMgU2NoZW1hXCIpXG4iLAogICAgIiAgICBvcHNfc2NoZW1hID0gc3QudGV4dF9pbnB1dChc
Ik9wZXJhdGlvbnMgU2NoZW1hXCIpXG4iLAogICAgIiAgICBpbnN0YWxsX3N0YWdlID0gc3QudGV4
dF9pbnB1dChcIkluc3RhbGwgU3RhZ2UgKGZvciBob3N0aW5nIGNvZGUgZmlsZXMpXCIpXG4iLAog
ICAgIiAgICBvcHNfd2ggPSBzdC50ZXh0X2lucHV0KFwiVXNlciBXYXJlaG91c2VcIikiCiAgIF0K
ICB9LAogIHsKICAgImNlbGxfdHlwZSI6ICJjb2RlIiwKICAgImV4ZWN1dGlvbl9jb3VudCI6IG51
bGwsCiAgICJpZCI6ICI4ZDUwY2JmNC0wYzhkLTQ5NTAtODZjYi0xMTQ5OTA0MzdhYzkiLAogICAi
bWV0YWRhdGEiOiB7CiAgICAiY29kZUNvbGxhcHNlZCI6IGZhbHNlLAogICAgImNvbGxhcHNlZCI6
IGZhbHNlLAogICAgImxhbmd1YWdlIjogInNxbCIsCiAgICAibmFtZSI6ICJwdXRfdG9fc3RhZ2Ui
LAogICAgInJlc3VsdEhlaWdodCI6IDExMgogICB9LAogICAib3V0cHV0cyI6IFtdLAogICAic291
cmNlIjogWwogICAgIkNSRUFURSBPUiBSRVBMQUNFIFBST0NFRFVSRSB7e2luc3RhbGxfZGJ9fS57
e2FkbWluX3NjaGVtYX19LlBVVF9UT19TVEFHRShTVEFHRSBWQVJDSEFSLCBGSUxFTkFNRSBWQVJD
SEFSLCBDT05URU5UIFZBUkNIQVIpXG4iLAogICAgIlJFVFVSTlMgU1RSSU5HXG4iLAogICAgIkxB
TkdVQUdFIFBZVEhPTlxuIiwKICAgICJSVU5USU1FX1ZFUlNJT049My44XG4iLAogICAgIlBBQ0tB
R0VTPSgnc25vd2ZsYWtlLXNub3dwYXJrLXB5dGhvbicpXG4iLAogICAgIkhBTkRMRVI9J3B1dF90
b19zdGFnZSdcbiIsCiAgICAiQ09NTUVOVD0ne1wib3JpZ2luXCI6IFwic2Zfc2l0XCIsXCJuYW1l
XCI6IFwic2l0X3NlcnZlbGVzc190YXNrX21pZ3JhdGlvblwiLFwidmVyc2lvblwiOiBcInttYWpv
cjogMSwgbWlub3I6IDB9XCJ9J1xuIiwKICAgICJBUyAkJFxuIiwKICAgICJpbXBvcnQgaW9cbiIs
CiAgICAiaW1wb3J0IG9zXG4iLAogICAgIlxuIiwKICAgICJkZWYgcHV0X3RvX3N0YWdlKHNlc3Np
b24sIHN0YWdlLCBmaWxlbmFtZSwgY29udGVudCk6XG4iLAogICAgIiAgICBsb2NhbF9wYXRoID0g
Jy90bXAnXG4iLAogICAgIiAgICBsb2NhbF9maWxlID0gb3MucGF0aC5qb2luKGxvY2FsX3BhdGgs
IGZpbGVuYW1lKVxuIiwKICAgICIgICAgZiA9IG9wZW4obG9jYWxfZmlsZSwgXCJ3XCIsIGVuY29k
aW5nPSd1dGYtOCcpXG4iLAogICAgIiAgICBmLndyaXRlKGNvbnRlbnQpXG4iLAogICAgIiAgICBm
LmNsb3NlKClcbiIsCiAgICAiICAgIHNlc3Npb24uZmlsZS5wdXQobG9jYWxfZmlsZSwgJ0AnK3N0
YWdlLCBhdXRvX2NvbXByZXNzPUZhbHNlLCBvdmVyd3JpdGU9VHJ1ZSlcbiIsCiAgICAiICAgIHJl
dHVybiBcInNhdmVkIGZpbGUgXCIrZmlsZW5hbWUrXCIgaW4gc3RhZ2UgXCIrc3RhZ2VcbiIsCiAg
ICAiJCQ7XG4iCiAgIF0KICB9LAogIHsKICAgImNlbGxfdHlwZSI6ICJjb2RlIiwKICAgImV4ZWN1
dGlvbl9jb3VudCI6IG51bGwsCiAgICJpZCI6ICIwNWM1OGMzOC0yOGRhLTQ1NjYtOTE2My02Mzdj
M2U1OTQ1YWEiLAogICAibWV0YWRhdGEiOiB7CiAgICAiY29kZUNvbGxhcHNlZCI6IGZhbHNlLAog
ICAgImNvbGxhcHNlZCI6IGZhbHNlLAogICAgImxhbmd1YWdlIjogInB5dGhvbiIsCiAgICAibmFt
ZSI6ICJlbmNvZGVkX29iamVjdCIsCiAgICAicmVzdWx0SGVpZ2h0IjogMAogICB9LAogICAib3V0
cHV0cyI6IFtdLAogICAic291cmNlIjogWwogICAgIlwiXCJcImdBU1ZIUUFBQUFBQUFBQjlsSXdG
Wm1sc1pYT1VmWlNNQ0M1bmFYUnJaV1Z3bEVNQWxITnpMZz09XG4iLAogICAgIlwiXCJcIiIKICAg
XQogIH0sCiAgewogICAiY2VsbF90eXBlIjogImNvZGUiLAogICAiZXhlY3V0aW9uX2NvdW50Ijog
bnVsbCwKICAgImlkIjogImM2OTUzNzNlLWFjNzQtNGI2Mi1hMWYxLTA4MjA2Y2JkNWM4MSIsCiAg
ICJtZXRhZGF0YSI6IHsKICAgICJjb2RlQ29sbGFwc2VkIjogZmFsc2UsCiAgICAiY29sbGFwc2Vk
IjogZmFsc2UsCiAgICAibGFuZ3VhZ2UiOiAicHl0aG9uIiwKICAgICJuYW1lIjogImRlY3J5cHRf
YW5kX2xvYWQiLAogICAgInJlc3VsdEhlaWdodCI6IDAKICAgfSwKICAgIm91dHB1dHMiOiBbXSwK
ICAgInNvdXJjZSI6IFsKICAgICJpbXBvcnQgcGlja2xlXG4iLAogICAgImltcG9ydCBjb2RlY3Nc
biIsCiAgICAiaW1wb3J0IG9zXG4iLAogICAgImltcG9ydCBiYXNlNjRcbiIsCiAgICAiXG4iLAog
ICAgImZyb20gc25vd2ZsYWtlLnNub3dwYXJrLmNvbnRleHQgaW1wb3J0IGdldF9hY3RpdmVfc2Vz
c2lvblxuIiwKICAgICJzZXNzaW9uID0gZ2V0X2FjdGl2ZV9zZXNzaW9uKClcbiIsCiAgICAiXG4i
LAogICAgImRlZiBkZWNvZGUob2JqKTpcbiIsCiAgICAiICAgIHBpY2tsZWQgPSBjb2RlY3MuZGVj
b2RlKG9iai5lbmNvZGUoKSwgJ2Jhc2U2NCcpXG4iLAogICAgIiAgICByZXR1cm4gcGlja2xlLmxv
YWRzKHBpY2tsZWQpXG4iLAogICAgIlxuIiwKICAgICJcbiIsCiAgICAiIyBEZWNvZGUgb2JqZWN0
IHRvIGNvbmZpcm0gaXQgaXMgdGhlIHNhbWUgYXMgdGhlIG9yaWdpbmFsIG9iamVjdFxuIiwKICAg
ICJkZWNvZGVkX29iaiA9IGRlY29kZShlbmNvZGVkX29iailcbiIsCiAgICAiXG4iLAogICAgIiMg
U3RhZ2UgZWFjaCBmaWxlXG4iLAogICAgImZvciBmaWxlX25hbWUsIGZpbGVfY29udGVudCBpbiBk
ZWNvZGVkX29ialsnZmlsZXMnXS5pdGVtcygpOlxuIiwKICAgICIgICAgIyByZXBsYWNlIGRvdWJs
ZSBkb2xsYXJzIHRvIGhlbHAgd2l0aCBTUUwgY29tcGF0aWJpbGl0eVxuIiwKICAgICIgICAgZmls
ZV9jb250ZW50ID0gc3RyKGZpbGVfY29udGVudC5kZWNvZGUoXCJ1dGYtOFwiKSkucmVwbGFjZShc
IiQkXCIsICdcIlwiXCInKVxuIiwKICAgICIgICAgc3FsID0gXCJcIlwiY2FsbCB7ZGJ9LntzY2hl
bWF9LnB1dF90b19zdGFnZSgne2RifS57c2NoZW1hfS57c3RhZ2V9Jywne2ZpbGVfbmFtZX0nLCAk
JHtmaWxlX2NvbnRlbnR9JCQpXCJcIlwiLmZvcm1hdChcbiIsCiAgICAiICAgICAgICBmaWxlX25h
bWU9ZmlsZV9uYW1lLCBcbiIsCiAgICAiICAgICAgICBmaWxlX2NvbnRlbnQ9ZmlsZV9jb250ZW50
LFxuIiwKICAgICIgICAgICAgIGRiPWluc3RhbGxfZGIsXG4iLAogICAgIiAgICAgICAgc2NoZW1h
PWFkbWluX3NjaGVtYSxcbiIsCiAgICAiICAgICAgICBzdGFnZT1pbnN0YWxsX3N0YWdlKVxuIiwK
ICAgICIgICAgc2Vzc2lvbi5zcWwoc3FsKS5jb2xsZWN0KClcbiIsCiAgICAiXG4iLAogICAgIiMg
U3RhZ2UgZWFjaCBpbWFnZSIKICAgXQogIH0sCiAgewogICAiY2VsbF90eXBlIjogImNvZGUiLAog
ICAiZXhlY3V0aW9uX2NvdW50IjogbnVsbCwKICAgImlkIjogImQwZmJkZTM4LTA0NDYtNDQyZS04
ZjdiLTdjZDQ0MjkzODBhOCIsCiAgICJtZXRhZGF0YSI6IHsKICAgICJjb2RlQ29sbGFwc2VkIjog
ZmFsc2UsCiAgICAiY29sbGFwc2VkIjogZmFsc2UsCiAgICAibGFuZ3VhZ2UiOiAic3FsIiwKICAg
ICJuYW1lIjogImluc3RhbGwiLAogICAgInJlc3VsdEhlaWdodCI6IDExMgogICB9LAogICAib3V0
cHV0cyI6IFtdLAogICAic291cmNlIjogWwogICAgIkNSRUFURSBPUiBSRVBMQUNFIFNUUkVBTUxJ
VCB7e2luc3RhbGxfZGJ9fS57e29wc19zY2hlbWF9fS5TVE1fU1RSRUFNTElUXG4iLAogICAgIlJP
T1RfTE9DQVRJT04gPSAnQHt7aW5zdGFsbF9kYn19Lnt7b3BzX3NjaGVtYX19Lnt7aW5zdGFsbF9z
dGFnZX19J1xuIiwKICAgICJNQUlOX0ZJTEUgPSAnc2VydmVybGVzc190YXNrX21pZ3JhdGlvbi5w
eSdcbiIsCiAgICAiUVVFUllfV0FSRUhPVVNFID0ge3tvcHNfd2h9fVxuIiwKICAgICJDT01NRU5U
PSd7XCJvcmlnaW5cIjogXCJzZl9zaXRcIixcIm5hbWVcIjogXCJzaXRfc2VydmVsZXNzX3Rhc2tf
bWlncmF0aW9uXCIsXCJ2ZXJzaW9uXCI6IFwie21ham9yOiAxLCBtaW5vcjogMH1cIn0nOyIKICAg
XQogIH0KIF0sCiAibWV0YWRhdGEiOiB7CiAgImtlcm5lbHNwZWMiOiB7CiAgICJkaXNwbGF5X25h
bWUiOiAiU3RyZWFtbGl0IE5vdGVib29rIiwKICAgIm5hbWUiOiAic3RyZWFtbGl0IgogIH0sCiAg
Imxhc3RFZGl0U3RhdHVzIjogewogICAiYXV0aG9yRW1haWwiOiAiaGFydGxhbmQuYnJvd25Ac25v
d2ZsYWtlLmNvbSIsCiAgICJhdXRob3JJZCI6ICIzMTMxNTcwNzAwMzg1IiwKICAgImF1dGhvck5h
bWUiOiAiSEJST1dOIiwKICAgImxhc3RFZGl0VGltZSI6IDE3NDYxMzM0NTUzNDQsCiAgICJub3Rl
Ym9va0lkIjogImlwaGRqNnpteGh4cmducTRxaXNoIiwKICAgInNlc3Npb25JZCI6ICJkNWVmYjZm
Ni1lYmU1LTRlYWQtYWVmMC1iZTc0ZTI2NWMwNTQiCiAgfQogfSwKICJuYmZvcm1hdCI6IDQsCiAi
bmJmb3JtYXRfbWlub3IiOiA1Cn0KlHVzLg==
"""

In [ ]:
import pickle
import codecs
import os
import base64

from snowflake.snowpark.context import get_active_session
session = get_active_session()

def decode(obj):
    pickled = codecs.decode(obj.encode(), 'base64')
    return pickle.loads(pickled)


# Decode object to confirm it is the same as the original object
decoded_obj = decode(encoded_obj)

# Stage each file
for file_name, file_content in decoded_obj['files'].items():
    # replace double dollars to help with SQL compatibility
    file_content = str(file_content.decode("utf-8")).replace("$$", '"""')
    sql = """call {db}.{schema}.put_to_stage('{db}.{schema}.{stage}','{file_name}', $${file_content}$$)""".format(
        file_name=file_name, 
        file_content=file_content,
        db=install_db,
        schema=admin_schema,
        stage=install_stage)
    session.sql(sql).collect()

# Stage each image

In [ ]:
CREATE OR REPLACE STREAMLIT {{install_db}}.{{ops_schema}}.STM_STREAMLIT
ROOT_LOCATION = '@{{install_db}}.{{ops_schema}}.{{install_stage}}'
MAIN_FILE = 'serverless_task_migration.py'
QUERY_WAREHOUSE = {{ops_wh}}
COMMENT='{"origin": "sf_sit","name": "sit_serveless_task_migration","version": "{major: 1, minor: 0}"}';